In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/final_data.csv")


In [ ]:
X = df.drop(columns=['price'])
y = df['price']

In [ ]:
pd.set_option("display.max.columns",None)

In [ ]:
df.head()

,brand,os,utility,warranty,display_size,ppi,aspect_ratio,antiglare,touch_screen,ram,hdd,ssd,graphic,cache,core,hdmi,mcr,wifi,backlit_keyboard,inbuilt_microphone,thunderbolt,fingerprint_sensor,ethernet,display_port,num_of_cell,battery_capacity,pixel_width,pixel_height,usb2,type_c,processor_brand,processor_model,processor_gen,price
0,2.0,2.0,1.0,1.0,2.0,157,1.0,1,0,8,0,512,1.0,6.0,2.0,1,0,1,0,1,0,0,1,0,2.0,45.500000,1920.0,1080.0,1,0,2.0,18.0,11.0,25990.0
1,25.0,2.0,0.0,1.0,1.0,141,1.0,1,0,8,0,512,1.0,8.0,4.0,1,0,1,1,1,0,0,0,0,2.0,17.850000,1080.0,1920.0,1,0,2.0,19.0,11.0,34990.0
2,16.0,2.0,3.0,2.0,1.0,141,1.0,1,0,16,0,512,1.0,12.0,8.0,1,0,1,1,1,1,0,1,0,3.0,51.000000,1920.0,1080.0,0,0,2.0,19.0,12.0,49990.0
3,2.0,2.0,2.0,1.0,1.0,141,1.0,1,0,8,0,512,2.0,12.0,8.0,1,0,1,1,1,0,0,1,0,3.0,49.616667,1920.0,1080.0,0,0,2.0,19.0,13.0,79745.0
4,2.0,2.0,3.0,1.0,1.0,141,1.0,1,0,16,0,512,1.0,8.0,6.0,1,1,1,0,1,0,0,1,0,3.0,36.000000,1920.0,1080.0,1,0,0.0,4.0,5.0,35990.0


In [ ]:
df["brand"].value_counts()

,count
brand,
4.0,164
10.0,156
15.0,138
16.0,88
7.0,86
2.0,73
13.0,17
21.0,11
3.0,10


In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [ ]:
#columns_to_encode = ['brand', 'os', 'utility', 'graphic', 'processor_brand', 'processor_model',"processor_gen"]
columns_to_encode = ["brand", 'os', 'utility',"display_size","aspect_ratio", 'graphic',"num_of_cell","usb2", 'processor_brand',"processor_gen","processor_model","type_c"]


In [ ]:
y_transformed = np.log1p(y)


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first',handle_unknown="ignore"), columns_to_encode),
        ('num', StandardScaler(), ["ppi","ram","hdd","ssd","cache","core","battery_capacity","pixel_width","pixel_height"])
    ],
    remainder='passthrough'
)

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel="rbf"))
])

In [ ]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [0, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown 

In [ ]:
scores.mean()

0.9001335077669734

In [ ]:
scores.std()

0.02243317711136003

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [ ]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['brand', 'os', 'utility',
                                                   'display_size',
                                                   'aspect_ratio', 'graphic',
                                                   'num_of_cell', 'usb2',
                                                   'processor_brand',
                                                   'processor_gen',
                                                   'processor_model',
                                                   'type_c']),
                                                 ('num', StandardScaler(),
                                                  ['ppi', 'ram', 'hdd', 'ssd',
                                                   'cache', 'core',
                                                   'battery_capacity',
                                                   'pixel_width',
                                                   'pixel_height'])])),
                ('regressor', SVR())])

In [ ]:
y_pred = pipeline.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [0, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
y_pred = np.expm1(y_pred)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

10565.388194970734